In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [ ]:
from dotenv import load_dotenv
load_dotenv('../.env')

AL_SUBMIT_API_KEY = os.getenv('AL_SUBMIT_API_KEY')
AL_USER = os.getenv('AL_USER')

In [ ]:
import hashlib
import os

def calculateFileSha256(filename):
    with open(filename,"rb") as f:
        bytes = f.read() # read entire file as bytes
        readable_hash = hashlib.sha256(bytes).hexdigest();
        return readable_hash

def submitFileToAL(filename):
    from assemblyline_client import get_client
    al_client = get_client("https://malware.cyber.gc.ca:443", apikey=(AL_USER, AL_SUBMIT_API_KEY))

    with open(filename,"rb") as f:
        bytes = f.read()
        submit_result = al_client.submit(fname=filename, content=bytes)
    
    return submit_result

In [ ]:
tools = [
    Tool(
        name = "CalculateFileSha256",
        func=calculateFileSha256,
        description="useful for when you need to calculate the sha256 of a file"
    ),
    Tool(
        name = "SubmitFileToAssemblyLine",
        func=submitFileToAL,
        description="useful for when you want to submit a file, using its filename, to the malware analysis platform AssemblyLine"
    )
]

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=ChatOpenAI(model="gpt-3.5-turbo-1106")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [ ]:
agent_chain.run("what is the sha256 of file benign_script.sh")

In [ ]:
agent_chain.run("please submit this file to AssemblyLine")

In [ ]:
agent_chain.run("please submit file mydoc.docx to AssemblyLine")